<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/usecases/10k_sub_question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solariix demo for 10-K Analysis of the 4 largest US banks
Answer complex queries by decomposing them into simpler sub-queries.

On colab: install LlamaIndex 🦙.

In [1]:
%pip install llama-index-llms-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.5 MB/s eta 0:00:00


In [2]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 23.8 MB/s eta 0:00:00


In [3]:
# patch asyncio to allow nested event loops.
import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

## Configure LLM service

In [5]:
# insure valid openai api key is defined
import openai
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

In [6]:
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")

## Connect Drive

In [7]:
# insure you have the the 10-k pdfs the dirs specified
from google.colab import drive
import os
drive.mount('/content/drive')

drive_path = '/content/drive/My Drive/UCF/SP24/FIN6777'
files = os.listdir(drive_path)
for file in files:
    print(file)





Mounted at /content/drive
jpmc-10k-2023.pdf
citi-10k-2023.pdf
boa-10k-2023.pdf
wf-10k-2023.pdf
ipynb


## Load 10-K for top 4 US banks

In [8]:
boa_docs = SimpleDirectoryReader(
    input_files=["/content/drive/My Drive/UCF/SP24/FIN6777/boa-10k-2023.pdf"]
).load_data()

citi_docs = SimpleDirectoryReader(
    input_files=["/content/drive/My Drive/UCF/SP24/FIN6777/citi-10k-2023.pdf"]
).load_data()
wf_docs = SimpleDirectoryReader(
    input_files=["/content/drive/My Drive/UCF/SP24/FIN6777/wf-10k-2023.pdf"]
).load_data()
jpmc_docs = SimpleDirectoryReader(
    input_files=["/content/drive/My Drive/UCF/SP24/FIN6777/jpmc-10k-2023.pdf"]
).load_data()

## Build indices

In [9]:
citi_index = VectorStoreIndex.from_documents(citi_docs)

In [10]:
wf_index = VectorStoreIndex.from_documents(wf_docs)

In [11]:
boa_index = VectorStoreIndex.from_documents(boa_docs)

In [12]:
jpmc_index = VectorStoreIndex.from_documents(jpmc_docs)

## Build query engines

In [13]:
jpmc_engine = jpmc_index.as_query_engine(similarity_top_k=3)

In [14]:
boa_engine = boa_index.as_query_engine(similarity_top_k=3)

In [15]:
citi_engine = citi_index.as_query_engine(similarity_top_k=3)

In [16]:
wf_engine = wf_index.as_query_engine(similarity_top_k=3)

In [17]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=jpmc_engine,
        metadata=ToolMetadata(
            name="jpmc_10k",
            description=(
                "Provides information about JPMC financials for year 2023"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=boa_engine,
        metadata=ToolMetadata(
            name="boa_10k",
            description=(
                "Provides information about Bank of America financials for year 2023"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=wf_engine,
        metadata=ToolMetadata(
            name="wf_10k",
            description=(
                "Provides information about Well Fargo financials for year 2023"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=citi_engine,
        metadata=ToolMetadata(
            name="citi_10k",
            description=(
                "Provides information about Citigroup financials for year 2023"
            ),
        ),
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools
)

## Run queries

In [18]:
response = s_engine.query(
    "How did the strategic priorities outlined in the 2023 10-K reports of the top 4 US banks align with their financial performance outcomes, and what does this reveal about the evolving landscape of the financial industry"

)

Generated 8 sub questions.
[jpmc_10k] Q: What were the strategic priorities outlined in the 2023 10-K report of JPMC?
[jpmc_10k] Q: How did the financial performance outcomes of JPMC in 2023 compare to the strategic priorities outlined in their 10-K report?
[boa_10k] Q: What were the strategic priorities outlined in the 2023 10-K report of Bank of America?
[boa_10k] Q: How did the financial performance outcomes of Bank of America in 2023 compare to the strategic priorities outlined in their 10-K report?
[wf_10k] Q: What were the strategic priorities outlined in the 2023 10-K report of Well Fargo?
[wf_10k] Q: How did the financial performance outcomes of Well Fargo in 2023 compare to the strategic priorities outlined in their 10-K report?
[citi_10k] Q: What were the strategic priorities outlined in the 2023 10-K report of Citigroup?
[citi_10k] Q: How did the financial performance outcomes of Citigroup in 2023 compare to the strategic priorities outlined in their 10-K report?
[jpmc_10k] 

In [19]:
import textwrap

max_line_length = 80
processed_response = textwrap.wrap(response.response, width=max_line_length, break_long_words=False, break_on_hyphens=False)
print('\n'.join(processed_response))


The strategic priorities outlined in the 2023 10-K reports of the top 4 US banks
aligned with their financial performance outcomes to varying degrees. While some
banks showed alignment between their strategic priorities and financial results,
others faced challenges in fully realizing their outlined priorities. This
reveals that the evolving landscape of the financial industry demands a careful
balance between strategic planning and operational execution. The ability to
effectively manage risks, control expenses, innovate, attract talent, and adapt
to changing market conditions is crucial for banks to navigate the complexities
of the financial industry and achieve sustainable growth and profitability.


In [19]:
response = s_engine.query(
    "Compare revenue growth of the two largest US banks from 2022 to 2023"
)

Generated 4 sub questions.
[jpmc_10k] Q: What was the revenue of JPMC in 2022?
[jpmc_10k] Q: What was the revenue of JPMC in 2023?
[boa_10k] Q: What was the revenue of Bank of America in 2022?
[boa_10k] Q: What was the revenue of Bank of America in 2023?
[boa_10k] A: Bank of America's revenue in 2022 was $21,748 million.
[jpmc_10k] A: The revenue of JPMC in 2023 was $70.1 billion.
[boa_10k] A: The revenue of Bank of America in 2023 was $21,105 million.
[jpmc_10k] A: The revenue of JPMC in 2022 was $48.8 billion.


In [20]:
import textwrap
max_line_length = 80
processed_response = textwrap.wrap(response.response, width=max_line_length, break_long_words=False, break_on_hyphens=False)
print('\n'.join(processed_response))

The revenue growth of JPMC from 2022 to 2023 was $21.3 billion, while the
revenue growth of Bank of America during the same period was a decrease of $643
million.


In [22]:
response = s_engine.query(
    "Can you list the greatest risk the four largest banks face collectively and independently"
)

Generated 5 sub questions.
[jpmc_10k] Q: What are the greatest risks faced collectively by JPMC, Bank of America, Wells Fargo, and Citigroup?
[jpmc_10k] Q: What are the greatest risks faced independently by JPMC?
[boa_10k] Q: What are the greatest risks faced independently by Bank of America?
[wf_10k] Q: What are the greatest risks faced independently by Wells Fargo?
[citi_10k] Q: What are the greatest risks faced independently by Citigroup?
[citi_10k] A: The greatest risks faced independently by Citigroup include credit risk, operational risk (specifically cybersecurity risk), and regulatory risk.
[wf_10k] A: The greatest risks faced independently by Wells Fargo include operational risks related to reliance on third-party service providers, potential disruptions in service, failure to meet regulatory requirements, and cybersecurity threats. Additionally, risks related to mortgage servicing obligations, competitive pressures in the financial services industry, challenges in attracting 

In [23]:
max_line_length = 80
processed_response = textwrap.wrap(response.response, width=max_line_length, break_long_words=False, break_on_hyphens=False)
print('\n'.join(processed_response))

Collectively, the greatest risks faced by JPMC, Bank of America, Wells Fargo,
and Citigroup include concentration or contagion risks, consumer credit
expansion risks, market and credit risks during dislocations, liquidity
constraints, capital risks, operational risks, strategic risks, conduct risks,
reputation risks, intra-day credit risks, risks associated with transactions
with government entities, and disputes with counterparties to derivatives
contracts. Independently, JPMC faces regulatory risks, political risks, market
risks, credit risks, liquidity risks, capital risks, operational risks,
strategic risks, conduct risks, and reputation risks. Bank of America faces
risks related to economic disruptions, credit losses, regulatory responses,
changes in accounting standards, cybersecurity incidents, geopolitical events,
climate change impacts, health emergencies, and natural disasters. Wells Fargo's
greatest risks include operational risks related to reliance on third-party
service p

In [24]:
response = s_engine.query(
    "Based on your response to the list of the greatest risk the four largest banks face can you list the top 3 risks in  that these banks face moving forward"
)

Generated 4 sub questions.
[jpmc_10k] Q: What are the top 3 risks that JPMC faces moving forward?
[boa_10k] Q: What are the top 3 risks that Bank of America faces moving forward?
[wf_10k] Q: What are the top 3 risks that Wells Fargo faces moving forward?
[citi_10k] Q: What are the top 3 risks that Citigroup faces moving forward?
[jpmc_10k] A: The top 3 risks that JPMorgan Chase (JPMC) faces moving forward are regulatory risks, market risks, and credit risks.
[wf_10k] A: Wells Fargo faces the top 3 risks moving forward related to its mortgage servicing obligations, competitive landscape in the financial services industry, and potential cyber attacks or information security incidents.
[citi_10k] A: The top 3 risks that Citigroup faces moving forward are regulatory scrutiny and legal proceedings, risks associated with its presence in emerging markets, and the financial and non-financial risks related to climate change.
[boa_10k] A: Bank of America faces risks related to credit, economic a

In [25]:
max_line_length = 80
processed_response = textwrap.wrap(response.response, width=max_line_length, break_long_words=False, break_on_hyphens=False)
print('\n'.join(processed_response))

The top 3 risks that the four largest banks (JPMorgan Chase, Bank of America,
Wells Fargo, and Citigroup) face moving forward are regulatory risks, market
risks, and credit risks.
